In [58]:
import time
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from datetime import datetime

# bill counter
global saved_value_index, value, END_REVENUE, END_no_TRANSACTIONS #, temp_saved_value_index
saved_value_index = [4, 3, 8, 2, 3, 13, 21]
#saved_value_index = [2, 1, 0, 0, 0, 0, 0]
value = (2000, 500, 200, 100, 50, 20, 10)
END_REVENUE = int(0)
END_no_TRANSACTIONS = int(0)

# environment data
global SUSPECT_IMAGE_NUMBER, SUSPECT_IMAGE_ADDRESS
SUSPECT_IMAGE_NUMBER = 0
SUSPECT_IMAGE_ADDRESS = ""

# SQL
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="kevindaniel",
  password="123456789"
)
mycursor = mydb.cursor()
#mycursor.execute("CREATE DATABASE mydatabase")
mydb = mysql.connector.connect(
  host="localhost",
  user="kevindaniel",
  password="123456789",
  database="mydatabase"
)
mycursor = mydb.cursor()

# LEGITIMATE
mycursor.execute("CREATE TABLE trial (ROW_NO int NOT NULL AUTO_INCREMENT, TwoThousand VARCHAR(255), FiveHundred VARCHAR(255), TwoHundred VARCHAR(255), Hundred VARCHAR(255), Fifty VARCHAR(255), Twenty VARCHAR(255), Ten VARCHAR(255), PRIMARY KEY (ROW_NO))")

# ILLICIT
mycursor.execute("CREATE TABLE trial_illicit (ROW_NO int NOT NULL AUTO_INCREMENT, TwoThousand VARCHAR(255), FiveHundred VARCHAR(255), TwoHundred VARCHAR(255), Hundred VARCHAR(255), Fifty VARCHAR(255), Twenty VARCHAR(255), Ten VARCHAR(255), PRIMARY KEY (ROW_NO))")

# TRANSACTION RECORD
mycursor.execute("CREATE TABLE Transation_log (ROW_NO int NOT NULL AUTO_INCREMENT, Location VARCHAR(255), Date VARCHAR(255), Time VARCHAR(255), BillValue_ByMerchant INT, TotalValue_ByCustomer INT, ImageAddress VARCHAR(255), SuspeciousCondition VARCHAR(255), StollenSerialNumberFound VARCHAR(255), TransactionState VARCHAR(255), PRIMARY KEY (ROW_NO))")

# For Legitimate table
global SQL_data_insertion_log, SQL_total_row_count
SQL_data_insertion_log = [0, 0, 0, 0, 0, 0, 0] #PER COLUMN
SQL_total_row_count = 0                        #OVERALL ROW

# For Transaction Table
Transaction_Log_Column_Name = ["Location", "Date", "Time", "BillValue_ByMerchant", "TotalValue_ByCustomer", "ImageAddress", "SuspeciousCondition", "StollenSerialNumberFound", "TransactionState"]
str_Transaction_Log_Column_Name = ""
for i in range(len(Transaction_Log_Column_Name)):
    if i==8:
        str_Transaction_Log_Column_Name = (str_Transaction_Log_Column_Name + str(Transaction_Log_Column_Name[i]))        
    else:
        str_Transaction_Log_Column_Name = (str_Transaction_Log_Column_Name + str(Transaction_Log_Column_Name[i] + ", "))
        
print(str_Transaction_Log_Column_Name)

global TOTAL_NO_of_RECORD_in_TRANSACTION_LOG
TOTAL_NO_of_RECORD_in_TRANSACTION_LOG = 0

Location, Date, Time, BillValue_ByMerchant, TotalValue_ByCustomer, ImageAddress, SuspeciousCondition, StollenSerialNumberFound, TransactionState


# ENVIRONMENT DATA

In [59]:
global SUSPECT_IMAGE_NUMBER
SUSPECT_IMAGE_NUMBER = 0
def ENVIRONMENT_DATA_GATHERING(suspect_condition):
    def location():
        shop_code = "001"
        area = "Kolathur"
        district = "Chennai"
        state = "TamilNadu"
        pin = "600099"
        address_location = shop_code + "|" + area + "|" + district + "|" + state + "|" + pin
        return address_location
        
    def date_time():
        initiate = datetime.now()
        current_date = initiate.strftime("%d:%m:%Y:%A")
        current_time = initiate.strftime("%H:%M:%S:%p")
        return current_date, current_time
    
    # IMAGE
    def image_capture():
        videoCaptureObject = cv2.VideoCapture(0)
        condition = True
        while(condition):
            ret,one_frame = videoCaptureObject.read()
            cv2.imwrite("Suspect_Image.jpg",one_frame)
            condition = False
        videoCaptureObject.release()
        cv2.destroyAllWindows()
        os.remove("C:/Users/kevin/Suspect_Image.jpg")    
        return one_frame
    
    def suspect_image_address(captured_image):
        global SUSPECT_IMAGE_NUMBER
        temp_image_address_holder = "C:/Users/kevin/Desktop/TEST/SUSPECT_NO_" + str((SUSPECT_IMAGE_NUMBER)+1) + ".jpeg"
        cv2.imwrite(temp_image_address_holder, captured_image)
        SUSPECT_IMAGE_NUMBER = SUSPECT_IMAGE_NUMBER + 1
        SUSPECT_IMAGE_ADDRESS = ""
        SUSPECT_IMAGE_ADDRESS = temp_image_address_holder
        return SUSPECT_IMAGE_ADDRESS
    
    if suspect_condition == False:   
        date, time = date_time()
        address = location()
        return date, time, None, address, "None"
    
    elif suspect_condition == True:
        date, time = date_time()
        captured_image = image_capture()
        address = location()
        image_address = suspect_image_address(captured_image)
        return date, time, captured_image, address, image_address

# SQL

In [60]:
def SQL_DATABASE_CONNECT(INPUT_SERIAL_NUMBER_PARAMETER, INPUT_DENOMINATION_PARAMETER, DISPENSED_PARAMETER):
    
    stollen_serialnumber_match_holder = [] # to hold stollen serial number
    def SQL_illicit_record(SNO, Denomination):
        STOLLEN_CONDITION = 0
        for i in range(len(Denomination)):
            mycursor.execute("SELECT " + Denomination[i] + " FROM trial_illicit")
            myresult = mycursor.fetchall()
            for x in myresult:
                check_result = (x[0] == SNO[i])
                if check_result == True:
                    STOLLEN_CONDITION = STOLLEN_CONDITION + 1
                    stollen_serialnumber_match_holder.append([x[0], Denomination[i]])
                else:
                    pass
        return STOLLEN_CONDITION
    
    stollen_bank_notes = SQL_illicit_record(INPUT_SERIAL_NUMBER_PARAMETER, INPUT_DENOMINATION_PARAMETER)
    
    if stollen_bank_notes == 0:      
        def SQL_insert(SNO, DENOMINATION):
            global SQL_data_insertion_log, SQL_total_row_count
            for i in range(len(SNO)):
                if DENOMINATION[i] == 'TwoThousand':
                    if SQL_total_row_count == SQL_data_insertion_log[0]:
                        query = "INSERT INTO trial (" + DENOMINATION[i] +") VALUES ('" + SNO[i] +"')"
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print("record inserted, ID:", mycursor.lastrowid)
                        SQL_data_insertion_log[0] = SQL_data_insertion_log[0] + 1
                        SQL_total_row_count = SQL_total_row_count + 1
            
                    else:
                        query = "UPDATE trial SET " + DENOMINATION[i] + " = '" + SNO[i] + "' WHERE ROW_NO = " + str(SQL_data_insertion_log[0]+1)
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print(mycursor.rowcount, "record(s) affected")
                        SQL_data_insertion_log[0] = SQL_data_insertion_log[0] + 1
                        
                elif DENOMINATION[i] == 'FiveHundred':
                    if SQL_total_row_count == SQL_data_insertion_log[1]:
                        query = "INSERT INTO trial (" + DENOMINATION[i] +") VALUES ('" + SNO[i] +"')"
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print("record inserted, ID:", mycursor.lastrowid)
                        SQL_data_insertion_log[1] = SQL_data_insertion_log[1] + 1
                        SQL_total_row_count = SQL_total_row_count + 1
            
                    else:
                        query = "UPDATE trial SET " + DENOMINATION[i] + " = '" + SNO[i] + "' WHERE ROW_NO = " + str(SQL_data_insertion_log[1]+1)
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print(mycursor.rowcount, "record(s) affected")
                        SQL_data_insertion_log[1] = SQL_data_insertion_log[1] + 1
                        
                elif DENOMINATION[i] == 'TwoHundred':
                    if SQL_total_row_count == SQL_data_insertion_log[2]:
                        query = "INSERT INTO trial (" + DENOMINATION[i] +") VALUES ('" + SNO[i] +"')"
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print("record inserted, ID:", mycursor.lastrowid)
                        SQL_data_insertion_log[2] = SQL_data_insertion_log[2] + 1
                        SQL_total_row_count = SQL_total_row_count + 1
            
                    else:
                        query = "UPDATE trial SET " + DENOMINATION[i] + " = '" + SNO[i] + "' WHERE ROW_NO = " + str(SQL_data_insertion_log[2]+1)
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print(mycursor.rowcount, "record(s) affected")
                        SQL_data_insertion_log[2] = SQL_data_insertion_log[2] + 1
                        
                elif DENOMINATION[i] == 'Hundred':
                    if SQL_total_row_count == SQL_data_insertion_log[3]:
                        query = "INSERT INTO trial (" + DENOMINATION[i] +") VALUES ('" + SNO[i] +"')"
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print("record inserted, ID:", mycursor.lastrowid)
                        SQL_data_insertion_log[3] = SQL_data_insertion_log[3] + 1
                        SQL_total_row_count = SQL_total_row_count + 1
            
                    else:
                        query = "UPDATE trial SET " + DENOMINATION[i] + " = '" + SNO[i] + "' WHERE ROW_NO = " + str(SQL_data_insertion_log[3]+1)
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print(mycursor.rowcount, "record(s) affected")
                        SQL_data_insertion_log[3] = SQL_data_insertion_log[3] + 1
                        
                elif DENOMINATION[i] == 'Fifty':
                    if SQL_total_row_count == SQL_data_insertion_log[4]:
                        query = "INSERT INTO trial (" + DENOMINATION[i] +") VALUES ('" + SNO[i] +"')"
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print("record inserted, ID:", mycursor.lastrowid)
                        SQL_data_insertion_log[4] = SQL_data_insertion_log[4] + 1
                        SQL_total_row_count = SQL_total_row_count + 1
            
                    else:
                        query = "UPDATE trial SET " + DENOMINATION[i] + " = '" + SNO[i] + "' WHERE ROW_NO = " + str(SQL_data_insertion_log[4]+1)
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print(mycursor.rowcount, "record(s) affected")
                        SQL_data_insertion_log[4] = SQL_data_insertion_log[4] + 1
                        
                elif DENOMINATION[i] == 'Twenty':
                    if SQL_total_row_count == SQL_data_insertion_log[5]:
                        query = "INSERT INTO trial (" + DENOMINATION[i] +") VALUES ('" + SNO[i] +"')"
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print("record inserted, ID:", mycursor.lastrowid)
                        SQL_data_insertion_log[5] = SQL_data_insertion_log[5] + 1
                        SQL_total_row_count = SQL_total_row_count + 1
            
                    else:
                        query = "UPDATE trial SET " + DENOMINATION[i] + " = '" + SNO[i] + "' WHERE ROW_NO = " + str(SQL_data_insertion_log[5]+1)
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print(mycursor.rowcount, "record(s) affected")
                        SQL_data_insertion_log[5] = SQL_data_insertion_log[5] + 1
                        
                elif DENOMINATION[i] == 'Ten':
                    if SQL_total_row_count == SQL_data_insertion_log[6]:
                        query = "INSERT INTO trial (" + DENOMINATION[i] +") VALUES ('" + SNO[i] +"')"
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print("record inserted, ID:", mycursor.lastrowid)
                        SQL_data_insertion_log[6] = SQL_data_insertion_log[6] + 1
                        SQL_total_row_count = SQL_total_row_count + 1
            
                    else:
                        query = "UPDATE trial SET " + DENOMINATION[i] + " = '" + SNO[i] + "' WHERE ROW_NO = " + str(SQL_data_insertion_log[6]+1)
                        query_str = str(query)
                        mycursor.execute(query_str)
                        mydb.commit()
                        print(mycursor.rowcount, "record(s) affected")
                        SQL_data_insertion_log[6] = SQL_data_insertion_log[6] + 1
                else:
                    pass
                        
        
        global column_name
        column_name = ['TwoThousand', 'FiveHundred', 'TwoHundred', 'Hundred', 'Fifty', 'Twenty', 'Ten']
        
        def SQL_delete(delete):
            total_deletion = 0
            for i in range(len(delete)):
                total_deletion = total_deletion + delete[i]
            
            if total_deletion > 0:
                for i in range(len(delete)):
                    if delete[i] > 0:
                        for j in range(delete[i]):
                            #query = "UPDATE trial SET TwoThousand = null WHERE ROW_NO = " + str(SQL_data_insertion_log[0])
                            query = "UPDATE trial SET " + column_name[i] + " = null WHERE ROW_NO = " + str(SQL_data_insertion_log[i])
                            query_str = str(query)
                            mycursor.execute(query_str)
                            mydb.commit()
                            SQL_data_insertion_log[i] = SQL_data_insertion_log[i] - 1
                            print(mycursor.rowcount, "record(s) deleted")
                    else:
                        pass
            else:
                pass
        
        SQL_insert(INPUT_SERIAL_NUMBER_PARAMETER, INPUT_DENOMINATION_PARAMETER) # Function Call
        SQL_delete(DISPENSED_PARAMETER) #Function Call
        
    if stollen_bank_notes == 0:
        print("SQL record operations success")
        return False, 0
    else:
        print(stollen_bank_notes, "Stollen Banknotes Found")
        return True, stollen_serialnumber_match_holder

In [61]:
TOTAL_NO_of_RECORD_in_TRANSACTION_LOG = 0
def TRANSACTION_LOG(STOLLEN_CONDITION, STOLLEN_SERIAL_NUMBER, current_date, current_time, billing_value, total_value_of_bank_notes_given, temp_image_address_holder):
    if STOLLEN_CONDITION == True:
        stollen_serialno_SQL_data = ""
        for i in range(len(STOLLEN_SERIAL_NUMBER)):
            temp = str(STOLLEN_SERIAL_NUMBER[i][0]) + " | " + str(STOLLEN_SERIAL_NUMBER[i][1] + ", ")
            stollen_serialno_SQL_data = stollen_serialno_SQL_data + temp    

    Transaction_data = ""
    
    if STOLLEN_CONDITION == True:
        data_to_be_recorded = ["001|Kolathur|Chennai|TamilNadu|600099", str(current_date), str(current_time), billing_value, total_value_of_bank_notes_given, str(temp_image_address_holder), "Suspecious", str(stollen_serialno_SQL_data), "Failed"]
        Transaction_data = data_to_be_recorded
    elif STOLLEN_CONDITION == False:
        data_to_be_recorded = ["001|Kolathur|Chennai|TamilNadu|600099", str(current_date), str(current_time), billing_value, total_value_of_bank_notes_given, "None", "Not-Suspecious", "None", "Success"]
        Transaction_data = data_to_be_recorded 
        
    def SQL_Transaction_Log(transaction_data):
        global TOTAL_NO_of_RECORD_in_TRANSACTION_LOG
        #query = "INSERT INTO Transation_log (" + str(str_Transaction_Log_Column_Name) + ") VALUES ('" + transaction_data[0] + "', '" + transaction_data[1] + "', '" + transaction_data[2]"', " + transaction_data[3] + ", " + transaction_data[4] + ", '" + transaction_data[5] + "', '" + transaction_data[6]"')" 
        query = "INSERT INTO Transation_log (" + str(str_Transaction_Log_Column_Name) + ") VALUES ('"+transaction_data[0]+ "','"+transaction_data[1]+ "','"+transaction_data[2]+ "',"+str(transaction_data[3])+ ","+str(transaction_data[4])+ ",'"+transaction_data[5]+ "','"+transaction_data[6]+ "','"+transaction_data[7]+ "','"+transaction_data[8]+ "')"   
        query_str = str(query)
        mycursor.execute(query_str)
        mydb.commit()
        print("record inserted, ID:", mycursor.lastrowid)
        
    SQL_Transaction_Log(Transaction_data)

In [62]:
a, b, c = ["stollen1","not_stollen2","stollen3"], ["TwoThousand", "FiveHundred", "Ten"], [0,0,0,0,0,0,0]
billing_value = 200
total_value_of_bank_notes_given = 200

In [70]:
STOLLEN_CONDITION, STOLLEN_SERIAL_NUMBER = SQL_DATABASE_CONNECT(a, b, c) # SQL data check and entry
#SQL_Transaction_Log(Transaction_data) # SQL transaction entry
current_date, current_time, suspect_image_cpatured, current_address, image_path = ENVIRONMENT_DATA_GATHERING(STOLLEN_CONDITION)
TRANSACTION_LOG(STOLLEN_CONDITION, STOLLEN_SERIAL_NUMBER, current_date, current_time, billing_value, total_value_of_bank_notes_given, image_path)

1 Stollen Banknotes Found
record inserted, ID: 4


In [71]:
print("ILLICIT______________________________________________________________\n")
# SELECT
mycursor.execute("SELECT * FROM trial_illicit")
myresult = mycursor.fetchall()
#print(myresult)
for x in myresult:
  print(x)

print("\nLEGITIMATE_________________________________________________________\n")
# SELECT
mycursor.execute("SELECT * FROM trial")
myresult = mycursor.fetchall()
for x in myresult:
  print(x)

print("\nDATA LOG___________________________________________________________\n")
print(SQL_data_insertion_log, SQL_total_row_count)

print("\nTRANSACTION LOG____________________________________________________\n")
# SELECT
print("Location, Date, Time, BillValue_ByMerchant, TotalValue_ByCustomer, ImageAddress, SuspeciousCondition, TransactionState")
mycursor.execute("SELECT * FROM Transation_log")
myresult = mycursor.fetchall()
for x in myresult:
  print(x)

ILLICIT______________________________________________________________

(1, 'stollen1', None, None, None, None, None, None)

LEGITIMATE_________________________________________________________

(1, 'stollen1', 'not_stollen2', None, None, None, None, 'stollen3')

DATA LOG___________________________________________________________

[1, 1, 0, 0, 0, 0, 1] 1

TRANSACTION LOG____________________________________________________

Location, Date, Time, BillValue_ByMerchant, TotalValue_ByCustomer, ImageAddress, SuspeciousCondition, TransactionState
(1, '001|Kolathur|Chennai|TamilNadu|600099', '25:10:2021:Monday', '21:20:32:PM', 200, 200, 'None', 'Not-Suspecious', 'None', 'Success')
(2, '001|Kolathur|Chennai|TamilNadu|600099', '25:10:2021:Monday', '21:20:56:PM', 200, 200, 'C:/Users/kevin/Desktop/TEST/SUSPECT_NO_1.jpeg', 'Suspecious', 'stollen1 | TwoThousand, ', 'Failed')
(3, '001|Kolathur|Chennai|TamilNadu|600099', '25:10:2021:Monday', '21:21:38:PM', 200, 200, 'C:/Users/kevin/Desktop/TEST/SUSPECT_

In [72]:
# Illicit
sql = "DROP TABLE trial_illicit"
mycursor.execute(sql)

# Legitimate 
sql = "DROP TABLE trial"
mycursor.execute(sql)

# Transaction
sql = "DROP TABLE Transation_log"
mycursor.execute(sql)

In [65]:
query = "INSERT INTO trial_illicit (TwoThousand) VALUES ('stollen1')"
query_str = str(query)
mycursor.execute(query_str)
mydb.commit()
print("record inserted, ID:", mycursor.lastrowid)

record inserted, ID: 1
